# Running the TTGammaProcessor

This cell will copy the test files from their location on eos to your local area.

In [ ]:
# #If you have not already done so, you can copy the files to test the code on from here
# # ONLY NEEDS TO BE DONE ONCE, CAN BE COMMENTED OUT WHEN YOU 
# !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/TTGamma_1l.root .
# !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/TTbar_1l.root .
# !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/WGamma.root .
# !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/ZGamma.root .
# !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/W4Jets.root .
# !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/ZJets.root .    

In [ ]:
from coffea import util, processor
from ttgamma import TTGammaProcessor


List of samples to be run on (fileset variable) and a dictionary containing the number of events processed for each sample

In [ ]:
fileset = {'TTGamma_SingleLept_2016':['TTGamma_1l.root'],
#            'TTbarPowheg_Semilept_2016':['TTbar_1l.root'],
#            'W4jets_2016':['W4Jets.root'],
#            'WGamma_01J_5f_2016':['WGamma.root'],
#            'ZGamma_01J_5f_lowMass_2016'['ZGamma.root'],
#            'DYjetsM50_2016':['ZJets.root'],
        }

mcEventYields = {'TTGamma_SingleLept_2016': 1000000.0,
                 'TTbarPowheg_Semilept_2016': 1000000.0,
                 'W4jets_2016': 1000000.0,
                 'WGamma_01J_5f_2016': 1000000.0,
                 'ZGamma_01J_5f_lowMass_2016': 1000000.0,
                 'DYjetsM50_2016': 1000000.0,
             }


Run the TTGammaProcessor on the list of files included in fileset.

You can specify the chunksize and maximum number of chunks to process from each sample (selecting a small number of events and one chunk will force coffea to process only a subset of the events for quicker debugging)

In [ ]:
import time
tstart = time.time()

output = processor.run_uproot_job(fileset,
                                  treename='Events',
                                  processor_instance=TTGammaProcessor(mcEventYields=mcEventYields),
                                  executor=processor.iterative_executor,
                                  executor_args={'workers': 4, 'flatten': True},
#                                   chunksize=1000,
#                                   maxchunks=1,
                                 )

elapsed = time.time() - tstart
print("Total time: %.1f seconds"%elapsed)
print("Total rate: %.1f events / second"%(output['EventCount'].value/elapsed))

In [ ]:
output

# Accessing Arrays Interactively

Below is an example of loading a JaggedCandidateArray interactively.  This can be very useful for developing the code, and debugging any issues

In [ ]:
import uproot
from coffea.analysis_objects import JaggedCandidateArray

In [ ]:
#Open the file TTGamma_1l.root, and access the Events TTree
_tree = uproot.open("TTGamma_1l.root")['Events']

In [ ]:
#print list of branches in the TTree
_tree.keys()

In [ ]:
#Load all branches which start with the word Jet
jetArray = _tree.arrays('Jet_*')
#get number of jets per event
nJets = _tree.array("nJet")

In [ ]:
jetArray.keys()

In [ ]:
jets = JaggedCandidateArray.candidatesfromcounts(nJets,
                                                 pt = jetArray[b'Jet_pt'].flatten(),
                                                 eta = jetArray[b'Jet_eta'].flatten(),
                                                 phi = jetArray[b'Jet_phi'].flatten(),
                                                 mass = jetArray[b'Jet_mass'].flatten(),
                                                )